In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import re
import csv
import pandas as pd

In [2]:
from os import walk

# folder path
dir_path = r'D:\Phase 5 project\reviews'
res = []
for (dir_path, dir_names, file_names) in walk(dir_path):
    # print(f'Adding these files: {file_names}')
    res.extend(file_names)
    # don't look inside any subdirectory
    break
print(res)

['reviews (1).csv', 'reviews (10).csv', 'reviews (11).csv', 'reviews (12).csv', 'reviews (13).csv', 'reviews (15).csv', 'reviews (3).csv', 'reviews (6).csv', 'reviews (7).csv', 'reviews (9).csv', 'reviews.csv', 'reviews2.csv', 'reviews3.csv']


In [3]:
# list of the files to keep
keep = []

for name in res:
    if 'review' in name:
        keep.append(name)
print(keep)

['reviews (1).csv', 'reviews (10).csv', 'reviews (11).csv', 'reviews (12).csv', 'reviews (13).csv', 'reviews (15).csv', 'reviews (3).csv', 'reviews (6).csv', 'reviews (7).csv', 'reviews (9).csv', 'reviews.csv', 'reviews2.csv', 'reviews3.csv']


In [4]:
# Confirms the list of files to pull data from
work_on = []
with open('list.csv', 'r') as csvfile:
    info = csvfile.read()
    for line in keep:
        if line in info:
            continue
        else:
            print(f'Pulling data from {line}')
            work_on.append(line)
    csvfile.close()        

In [5]:
li = []
with open('list.csv', 'a+', encoding='utf-8') as csvfile:
    list_writer = csv.writer(csvfile)
    dic = {}
    length = len(work_on)
    print(length)
    while length >= 1:
        for name in work_on:
            print(f'{name} is being read ..............')
            df = pd.read_csv(f'reviews/{name}', index_col=None, on_bad_lines='skip', header=None)
            li.append(df)
            dic['name'] = name
            list_writer.writerow(dic.values())
            length -= 1
        final_frame = pd.concat(li, axis=0, ignore_index=True)
        print('Gone through entire list')

0


In [6]:
final_frame.head()

NameError: name 'final_frame' is not defined

In [ ]:
# remove duplicates
final_frame = final_frame.drop_duplicates()

# check duplicates
print(f'The num of duplicates are: {final_frame.duplicated().sum()} duplicates')


In [ ]:
# columns having null values
final_frame.info()

In [ ]:
final_frame = final_frame.dropna(axis=0)
final_frame.info()

In [ ]:
# check the percentage of restaurants done in the final csv

restaurants = pd.read_csv('restaurant_data.csv')

rest_df = restaurants.drop_duplicates(subset=['URL'])
rest_df.head()

In [ ]:
# get the set of available restaurants
got_rest = set(final_frame[6])

list_rest = set(rest_df['URL'])

In [ ]:
# What restaurants do we have
missing_urls = []
for url in list_rest:
    if url in got_rest:
        num = len(final_frame[final_frame[6] == url])
        expected = rest_df[rest_df['URL'] == url]
        # print(expected['Number of Reviews'])
        eventual = float(num/expected['Number of Reviews']) *100
        if eventual >= 90.0:
            print(f"We have {expected['Name']} and have about ....... {eventual}% of expected collection")
            print('WITHIN COMPLETED EXPECTATION')
        else:
            print(f"We have {expected['Name']} and have about ....... {eventual}% of expected collection")
            print(f"{expected['Name']} NEEDS MORE COLLECTION PLEASE")
            missing_urls.append(url)
    else:
        expected = rest_df[rest_df['URL'] == url]
        num = 0
        eventual = (num/expected['Number of Reviews']) *100
        # print(f"We have {expected['Name']} and have about ....... {eventual}% of expected collection")
        # print(f"{expected['Name']} NEEDS MORE COLLECTION PLEASE")
        missing_urls.append(url)

        
print('Done with list of content')

In [ ]:
len(missing_urls)

In [ ]:
with open('final_revs.csv', 'a+', encoding='utf-8') as csvfile:
        review_writer = csv.writer(csvfile)
        for index, row in final_frame.iterrows():
            dic = {}
        
            # Assign the user values to the dictionary
            dic['username'] = row[0]
            dic['account'] = row[1]
            dic['location'] = row[2]
            dic['date'] = row[3]
            dic['rating'] = row[4]
            dic['comments'] = row[5]
            dic['restaurant'] = row[6]

            # write this info to the local csv
            review_writer.writerow(dic.values())
print('Done')

In [ ]:
try:
    missing_url_df = pd.read_csv('missing_urls.csv', header=None)
except pd.errors.EmptyDataError:
    found_missing_urls = list(['File is empty'])
else:
    found_missing_urls = list(missing_url_df)


found_missing_urls

In [ ]:
# Dealing with missing urls
with open('missing_urls.csv', 'a+', encoding='utf-8') as csvfile:
        review_writer = csv.writer(csvfile)
        for url in missing_urls:
            dic = {}
            # print(url)
        
            if url in found_missing_urls:
                continue
            else:
                # Assign the user values to the dictionary
                dic['url'] = url

                # write this info to the local csv
                review_writer.writerow(dic.values())
print('Done')